In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from statistics import mean 
import json
from datetime import datetime

In [2]:
right_json = gpd.read_file("right_sidewalk.geojson")
left_json = gpd.read_file("left_sidewalk.geojson")
right_metadata = pd.read_excel('right_sidewalk_metadata.xlsx')
left_metadata = pd.read_excel('left_sidewalk_metadata.xlsx')

road_grades = pd.read_csv('road_grades_by_segment.csv')

city_limits = gpd.read_file('citylimit.geojson')
city_polygon = city_limits['geometry'][0]

# there's just one row per object ID
roads_metadata = pd.read_excel('roads_metadata.xlsx')

In [3]:
right_json.shape

(30737, 9)

In [4]:
left_json.shape

(30932, 9)

In [5]:
def make_point(row, lat, long):
    return Point(row[long], row[lat])

def start_or_end_in_city_limits(row):
    return (row['startPoint'].within(city_polygon) or row['endPoint'].within(city_polygon))

def restrict_starting_file_to_sidewalks_in_city_limits(sidewalk_file):
    sidewalk_file = sidewalk_file.drop_duplicates()
    sidewalk_file['startPoint'] = sidewalk_file.apply(make_point, axis=1, args=('LatStart', 'LongStart',))
    sidewalk_file['endPoint'] = sidewalk_file.apply(make_point, axis=1, args=('LatEnd', 'LongEnd',))
    print("Number of sidewalks before removing those outside city limits: {}".format(sidewalk_file.shape))
    sidewalk_file = sidewalk_file[sidewalk_file.apply(start_or_end_in_city_limits, axis=1)]
    print("Number of sidewalks after removing those outside city limits: {}".format(sidewalk_file.shape))
    
    return sidewalk_file
    
def merge_with_roadgrades(sidewalk_file, roadgrades_file, whichArcgis):
    print("Dropping duplicates")
    roadgrades_file = roadgrades_file.drop_duplicates()
    
    sidewalk_segment_3 = sidewalk_file.merge(roadgrades_file, 
                                              left_on='road_OBJECTID', 
                                              right_on='road_objectID')
    print("Shape of sidewalk file after merging with elevations file (this limits to city only): {}".format(sidewalk_segment_3.shape))
    sidewalk_segment_3['whichArcgisFile'] = whichArcgis
    return sidewalk_segment_3


In [6]:
right_sidewalks = merge_with_roadgrades(right_json, road_grades, 'right')
left_sidewalks = merge_with_roadgrades(left_json, road_grades, 'left')

Dropping duplicates
Shape of sidewalk file after merging with elevations file (this limits to city only): (15575, 11)
Dropping duplicates
Shape of sidewalk file after merging with elevations file (this limits to city only): (15638, 11)


In [7]:
right_sidewalks['direction'] = ["unknown" for i in range(len(right_sidewalks))]
left_sidewalks['direction'] = ["unknown" for i in range(len(left_sidewalks))]

In [8]:
all_road_object_ids = list(right_sidewalks.road_objectID.values) + list(left_sidewalks.road_objectID.values)
all_road_object_ids = set(all_road_object_ids)

In [9]:
for i, road_object_id in enumerate(list(all_road_object_ids)):
    if i % 1000 == 0:
        print("Finished {}".format(i))
    try:       
        r_dir = ""
        l_dir = ""
        
        r_coord_list = list(right_sidewalks[right_sidewalks.road_objectID==road_object_id].geometry.values[0].coords)
        r_lat_avg = mean([a[1] for a in r_coord_list])
        r_long_avg = mean([a[0] for a in r_coord_list])
        
        l_coord_list = list(left_sidewalks[left_sidewalks.road_objectID==road_object_id].geometry.values[0].coords)
        l_lat_avg = mean([a[1] for a in l_coord_list])
        l_long_avg = mean([a[0] for a in l_coord_list])
        
        lat_dist = l_lat_avg - r_lat_avg
        long_dist = l_long_avg - r_long_avg

        if abs(lat_dist) > abs(long_dist):
            if lat_dist > 0:
                l_dir = "N"
                r_dir = "S"
            else:
                l_dir = "S"
                r_dir = "N"
        else:
            if long_dist > 0:
                # gets more negative as you move east
                # so if left long is greater than right long 
                # that means it's less negative
                # so left is west
                l_dir = "W"
                r_dir = "E"
            else:
                l_dir = "E"
                r_dir = "W"
                
        right_sidewalks.loc[right_sidewalks['road_objectID']==road_object_id, 'direction'] = r_dir
        left_sidewalks.loc[left_sidewalks['road_objectID']==road_object_id, 'direction'] = l_dir
    except Exception as ex:
        print("skipping road_object_id {}".format(road_object_id))
        print(ex)

Finished 0
skipping road_object_id 5189
index 0 is out of bounds for axis 0 with size 0
skipping road_object_id 5286
index 0 is out of bounds for axis 0 with size 0
skipping road_object_id 7529
index 0 is out of bounds for axis 0 with size 0
Finished 1000
skipping road_object_id 8067
Multi-part geometries do not provide a coordinate sequence
skipping road_object_id 8090
index 0 is out of bounds for axis 0 with size 0
skipping road_object_id 8512
index 0 is out of bounds for axis 0 with size 0
skipping road_object_id 8555
Multi-part geometries do not provide a coordinate sequence
Finished 2000
skipping road_object_id 9546
index 0 is out of bounds for axis 0 with size 0
skipping road_object_id 9950
index 0 is out of bounds for axis 0 with size 0
skipping road_object_id 10053
index 0 is out of bounds for axis 0 with size 0
skipping road_object_id 10126
index 0 is out of bounds for axis 0 with size 0
Finished 3000
skipping road_object_id 11148
Multi-part geometries do not provide a coordin

In [10]:
right_sidewalks.head()

,OBJECTID,FULLNAME,LatStart,LongStart,LatEnd,LongEnd,road_OBJECTID,Shape_Length,geometry,road_objectID,road_grade,whichArcgisFile,direction
0,3338,S 19th St,42.921256,-87.939263,42.920893,-87.939275,3371,0.000363,"LINESTRING (-87.93927 42.92126, -87.93927 42.9...",3371,0.013539,right,E
1,3346,W Aspen Dr,42.921360,-87.936933,42.921376,-87.937892,3379,0.000958,"LINESTRING (-87.93694 42.92137, -87.93707 42.9...",3379,0.013539,right,N
2,3348,W Aspen Dr,42.921379,-87.938105,42.921395,-87.939098,3381,0.000992,"LINESTRING (-87.93811 42.92139, -87.93825 42.9...",3381,0.016706,right,N
3,3370,S 13th St,42.922578,-87.930591,42.920652,-87.930658,3404,0.001928,"LINESTRING (-87.93060 42.92259, -87.93060 42.9...",3404,0.016706,right,E
4,3379,S 20th St,42.923533,-87.940446,42.920218,-87.940578,3413,0.003318,"LINESTRING (-87.94045 42.92354, -87.94047 42.9...",3413,0.009984,right,E


In [11]:
# MAKE SURE there are no overlaps of same road obj ID and sidewalk ID 
r_unknowns = list(right_sidewalks[right_sidewalks['direction']=='unknown'].road_objectID.values)
l_unknowns = list(left_sidewalks[left_sidewalks['direction']=='unknown'].road_objectID.values)
assert not set(r_unknowns).isdisjoint(l_unknowns)

In [38]:
## Make new dataframe, which is the one we'll actually import into sql 
df_to_load = pd.DataFrame()
df_to_load['segmentId'] = right_sidewalks['OBJECTID'].append(left_sidewalks['OBJECTID'])
df_to_load['directionInd'] = right_sidewalks['direction'].append(left_sidewalks['direction'])
df_to_load['linearId'] = right_sidewalks['road_objectID'].append(left_sidewalks['road_objectID'])
df_to_load['streetName'] = right_sidewalks['FULLNAME'].append(left_sidewalks['FULLNAME'])
df_to_load['startLat'] = right_sidewalks['LatStart'].append(left_sidewalks['LatStart'])
df_to_load['startLong'] = right_sidewalks['LongStart'].append(left_sidewalks['LongStart'])
df_to_load['endLat'] = right_sidewalks['LatEnd'].append(left_sidewalks['LatEnd'])
df_to_load['endLong'] = right_sidewalks['LongEnd'].append(left_sidewalks['LongEnd'])
df_to_load['whichArcgisFile'] = right_sidewalks['whichArcgisFile'].append(left_sidewalks['whichArcgisFile'])
df_to_load['roadGrade'] = right_sidewalks['road_grade'].append(left_sidewalks['road_grade'])
df_to_load['geometry'] = right_sidewalks['geometry'].append(left_sidewalks['geometry'])

In [39]:
df_to_load = df_to_load.reset_index()

In [40]:
# RESET SEGMENT ID to just be the index
df_to_load['segmentId'] = df_to_load.index.tolist()

In [41]:
def turn_to_geojson(row):
    init_json = gpd.GeoSeries([row['geometry']]).__geo_interface__
    feature_only = init_json['features'][0]
    # this is so that each different feature has a unique index
    feature_only['id'] = row.name
    return json.dumps(feature_only)

In [42]:
# keeping this one separate because it takes a little while 
df_to_load['geoJson'] = df_to_load.apply(turn_to_geojson, axis=1)

In [43]:
df_to_load['updateTs'] = datetime.now()

In [44]:
df_to_load = df_to_load.filter(['segmentId', 'directionInd', 'linearId', 'streetName', 'startLat', 'startLong', 'endLat', 'endLong', 'whichArcgisFile', 'roadGrade', 'geoJson'])

In [45]:
df_to_load.to_csv('sidewalk_segment3.csv')

In [46]:
df_to_load.head()

,segmentId,directionInd,linearId,streetName,startLat,startLong,endLat,endLong,whichArcgisFile,roadGrade,geoJson
0,0,E,3371,S 19th St,42.921256,-87.939263,42.920893,-87.939275,right,0.013539,"{""id"": 0, ""type"": ""Feature"", ""properties"": {},..."
1,1,N,3379,W Aspen Dr,42.921360,-87.936933,42.921376,-87.937892,right,0.013539,"{""id"": 1, ""type"": ""Feature"", ""properties"": {},..."
2,2,N,3381,W Aspen Dr,42.921379,-87.938105,42.921395,-87.939098,right,0.016706,"{""id"": 2, ""type"": ""Feature"", ""properties"": {},..."
3,3,E,3404,S 13th St,42.922578,-87.930591,42.920652,-87.930658,right,0.016706,"{""id"": 3, ""type"": ""Feature"", ""properties"": {},..."
4,4,E,3413,S 20th St,42.923533,-87.940446,42.920218,-87.940578,right,0.009984,"{""id"": 4, ""type"": ""Feature"", ""properties"": {},..."


In [47]:
import pymysql
connection = pymysql.connect(host='aa6f8tcntjfd0z.c5625ddefrth.us-west-2.rds.amazonaws.com',
                             port=3306,
                             user=#ask Emily :)  ,
                             password= #ask Emiliy :) ,
                             db='ebdb')
cursor=connection.cursor()

In [48]:
# NOTE: this method is way slow
# will take over an hour to load in 40,000 rows this way 
# creating column list for insertion
cols = "`,`".join([str(i) for i in df_to_load.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_to_load.iterrows():
    sql = "INSERT INTO `sidewalk_segment3` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()